In [1]:
pip install aif360['all']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 25.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
from xgboost import XGBClassifier

In [5]:
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric

/usr/local/lib/python3.11/dist-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
/usr/local/lib/python3.11/dist-packages/inFairness/utils/ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg = vmap(vect_normalized_discounted

In [6]:
from aif360.algorithms.preprocessing import (
    Reweighing,
    DisparateImpactRemover,
    LFR
)

In [7]:
from aif360.algorithms.inprocessing import (
    MetaFairClassifier,
    GerryFairClassifier,
    PrejudiceRemover,
    ExponentiatedGradientReduction,
    GridSearchReduction,
    ARTClassifier,
    AdversarialDebiasing
)

In [8]:
from aif360.algorithms.postprocessing import (
    RejectOptionClassification,
    CalibratedEqOddsPostprocessing,
    EqOddsPostprocessing
)

In [9]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [10]:
csv_path='/content/gmsc-training.csv'

In [11]:
def load_and_preprocess_gmsc_csv(csv_path):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler

    # Load the GMSC dataset
    df = pd.read_csv(csv_path)

    # Drop the unnecessary index column
    df = df.drop(columns=['Unnamed: 0'])

    # Fill missing values
    df['MonthlyIncome'] = df['MonthlyIncome'].fillna(df['MonthlyIncome'].median())
    df['NumberOfDependents'] = df['NumberOfDependents'].fillna(df['NumberOfDependents'].median())

    # Flip labels: 0 → 1 (Good), 1 → 0 (Bad)
    df['target'] = df['SeriousDlqin2yrs'].apply(lambda x: 1 if x == 1 else 0)
    df.drop(columns=['SeriousDlqin2yrs'], inplace=True)

    # Protected attribute: age >= 25 → privileged
    df['age_binary'] = df['age'].apply(lambda x: 1 if x >= 25 else 0)

    # Features
    df_features = df.drop(columns=['target', 'age', 'age_binary'])

    # Feature matrix (X), Labels (y), and Protected Attribute (protected)
    X = df_features
    y = df['target']
    protected = df['age_binary']

    # Standardize numeric features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Train-test split
    X_train, X_test, y_train, y_test, prot_train, prot_test = train_test_split(
        X_scaled, y, protected, test_size=0.3, random_state=42, stratify=y
    )

    return X_train, X_test, y_train, y_test, prot_train, prot_test


In [12]:
def load_gmsc_data(csv_path):
    import pandas as pd

    # Load dataset
    df = pd.read_csv(csv_path)

    # Drop unnecessary index column
    df = df.drop(columns=['Unnamed: 0'])

    # Fill missing values
    df['MonthlyIncome'] = df['MonthlyIncome'].fillna(df['MonthlyIncome'].median())
    df['NumberOfDependents'] = df['NumberOfDependents'].fillna(df['NumberOfDependents'].median())

    # Flip labels: 0 → 1 (Good), 1 → 0 (Bad)
    df['target'] = df['SeriousDlqin2yrs'].apply(lambda x: 1 if x == 1 else 0)
    df.drop(columns=['SeriousDlqin2yrs'], inplace=True)

    # Protected attribute: age >= 25 is privileged
    df['age_binary'] = df['age'].apply(lambda x: 1 if x >= 25 else 0)

    # Raw features (excluding target and protected columns)
    X_raw = df.drop(columns=['target', 'age', 'age_binary'])
    y = df['target'].values
    prot = df['age_binary'].values

    return X_raw, y, prot

def preprocess_data(X_raw, y, prot):
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    import numpy as np

    # GMSC dataset: only numeric columns
    num_cols = X_raw.columns.tolist()

    # Train-test split
    X_raw_train, X_raw_test, y_train, y_test, prot_train, prot_test = train_test_split(
        X_raw, y, prot, test_size=0.3, random_state=42, stratify=y
    )

    # No encoding needed (no categorical features)
    X_train_num = X_raw_train[num_cols].values
    X_test_num = X_raw_test[num_cols].values

    # Normalize
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train_num)
    X_test = scaler.transform(X_test_num)

    return X_train, X_test, y_train, y_test, prot_train, prot_test


In [13]:
def load_gmsc_data_inprocess(csv_path):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler

    # Load dataset
    df = pd.read_csv(csv_path)

    # Drop unnecessary index column
    df = df.drop(columns=['Unnamed: 0'])

    # Fill missing values
    df['MonthlyIncome'] = df['MonthlyIncome'].fillna(df['MonthlyIncome'].median())
    df['NumberOfDependents'] = df['NumberOfDependents'].fillna(df['NumberOfDependents'].median())

    # Flip labels: 0 → 1 (Good), 1 → 0 (Bad)
    df['target'] = df['SeriousDlqin2yrs'].apply(lambda x: 1 if x == 1 else 0)
    df.drop(columns=['SeriousDlqin2yrs'], inplace=True)

    # Protected attribute: age >= 25 is privileged
    df['age_binary'] = df['age'].apply(lambda x: 1 if x >= 25 else 0)

    # Features (drop target, age, and protected attribute)
    X_raw = df.drop(columns=['target', 'age', 'age_binary'])
    y = df['target'].values
    prot = df['age_binary'].values

    # Train-test split
    X_train_raw, X_test_raw, y_train, y_test, prot_train, prot_test = train_test_split(
        X_raw, y, prot, test_size=0.3, random_state=42, stratify=y
    )

    # GMSC dataset: all columns are numeric
    X_train_num = X_train_raw.values
    X_test_num = X_test_raw.values

    # Standardize all features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train_num)
    X_test = scaler.transform(X_test_num)

    return X_train, X_test, y_train, y_test, prot_train, prot_test


In [14]:
def train_baseline_model(X_train, y_train, sample_weight=None):
    model = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
    model.fit(X_train, y_train, sample_weight=sample_weight)
    return model

In [15]:
def evaluate_fairness(y_true, y_pred, prot, X=None):
    df = pd.DataFrame(np.hstack((X, y_true[:, None], prot[:, None])),
                      columns=[f"x{i}" for i in range(X.shape[1])] + ['label', 'protected'])

    dataset_true = BinaryLabelDataset(df=df,
                                      label_names=["label"],
                                      protected_attribute_names=["protected"],
                                      favorable_label=0, unfavorable_label=1)

    pred_dataset = dataset_true.copy()
    pred_dataset.labels = y_pred.reshape(-1, 1)

    metric = ClassificationMetric(dataset_true, pred_dataset,
                                  privileged_groups=[{'protected': 1}],
                                  unprivileged_groups=[{'protected': 0}])

    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'disparate_impact': metric.disparate_impact(),
        'statistical_parity_difference': metric.statistical_parity_difference(),
        'equal_opportunity_difference': metric.equal_opportunity_difference()
    }

In [16]:
def evaluate_model(model, X_test, y_test, prot_test):
    y_pred = model.predict(X_test)
    return evaluate_fairness(y_test, y_pred, prot_test, X_test)

In [19]:
#XGboost
X_train, X_test, y_train, y_test, prot_train, prot_test = load_and_preprocess_gmsc_csv(csv_path)
baseline_model = train_baseline_model(X_train, y_train)
baseline_metrics = evaluate_model(baseline_model, X_test, y_test.to_numpy(), prot_test.to_numpy())

baseline_metrics

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:30:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


{'accuracy': 0.9348,
 'disparate_impact': np.float64(0.9916306609265991),
 'statistical_parity_difference': np.float64(-0.008171566961345467),
 'equal_opportunity_difference': np.float64(-0.006311552664603592)}

In [20]:
##disparate impact
def apply_disparate_impact_remover(X_train, y_train, prot_train, repair_level=1.0):
    df = pd.DataFrame(X_train)
    df['target'] = y_train.values if hasattr(y_train, 'values') else y_train
    df['protected'] = prot_train.values if hasattr(prot_train, 'values') else prot_train

    dataset = BinaryLabelDataset(
        favorable_label=0,  # <-- GOOD outcome = 0 for GMSC
        unfavorable_label=1,
        df=df,
        label_names=['target'],
        protected_attribute_names=['protected']
    )

    dir_remover = DisparateImpactRemover(repair_level=repair_level)
    repaired_dataset = dir_remover.fit_transform(dataset)

    X_repaired = pd.DataFrame(repaired_dataset.features)
    y_repaired = pd.Series(repaired_dataset.labels.ravel())
    prot_repaired = pd.Series(repaired_dataset.protected_attributes.ravel())

    return X_repaired, y_repaired, prot_repaired

In [21]:
inprocess_results = {}

In [22]:
# Apply DIR ONLY to training data
X_train_repaired, y_train_repaired, prot_train_repaired = apply_disparate_impact_remover(
    pd.DataFrame(X_train), y_train, prot_train
)

# Split repaired training data into sub-train and validation
X_subtrain, X_val, y_subtrain, y_val, prot_subtrain, prot_val = train_test_split(
    X_train_repaired, y_train_repaired, prot_train_repaired, test_size=0.3, random_state=42, stratify=y_train_repaired
)

# Train model on sub-train
fair_model = train_baseline_model(X_subtrain, y_subtrain)

# Evaluate model on repaired validation
fair_metrics = evaluate_model(
    fair_model,
    X_val.to_numpy(),
    y_val.to_numpy(),
    prot_val.to_numpy()
)
inprocess_results['Disparate_Impact_Remover'] = fair_metrics
fair_metrics

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:30:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


{'accuracy': 0.9344761904761905,
 'disparate_impact': np.float64(0.9733450407231932),
 'statistical_parity_difference': np.float64(-0.026210380956125934),
 'equal_opportunity_difference': np.float64(-0.019605082167584387)}

In [23]:
##LFR

In [24]:
def apply_lfr(X_train, y_train, prot_train):
    df = pd.DataFrame(X_train)
    df['target'] = y_train.values
    df['protected'] = prot_train.values

    dataset = BinaryLabelDataset(
        favorable_label=0,
        unfavorable_label=1,
        df=df,
        label_names=['target'],
        protected_attribute_names=['protected']
    )

    lfr = LFR(unprivileged_groups=[{'protected': 0}],
          privileged_groups=[{'protected': 1}],
          k=10, Ax=0.01, Ay=1.0, Az=0.1, verbose=0)

    lfr.fit(dataset)
    transformed_dataset = lfr.transform(dataset)

    X_transformed = pd.DataFrame(transformed_dataset.features)
    y_transformed = pd.Series(transformed_dataset.labels.ravel())
    prot_transformed = pd.Series(transformed_dataset.protected_attributes.ravel())

    return X_transformed, y_transformed, prot_transformed, lfr

In [25]:
X_train_lfr, y_train_lfr, prot_train_lfr, lfr_model = apply_lfr(pd.DataFrame(X_train), y_train, prot_train)

In [26]:
print("Unique labels in y_train_lfr:", np.unique(y_train_lfr))

Unique labels in y_train_lfr: [0. 1.]


In [27]:
fair_lfr_model = train_baseline_model(X_train_lfr, y_train_lfr)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:38:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [28]:
# Apply LFR transformation on test set
df_test = pd.DataFrame(X_test)
df_test['target'] = y_test.values
df_test['protected'] = prot_test.values

test_dataset = BinaryLabelDataset(
    favorable_label=0,
    unfavorable_label=1,
    df=df_test,
    label_names=['target'],
    protected_attribute_names=['protected']
)

transformed_test_dataset = lfr_model.transform(test_dataset)

X_test_lfr = pd.DataFrame(transformed_test_dataset.features)
y_test_lfr = pd.Series(transformed_test_dataset.labels.ravel())
prot_test_lfr = pd.Series(transformed_test_dataset.protected_attributes.ravel())

# Evaluate
fair_lfr_metrics = evaluate_model(
    fair_lfr_model,
    X_test_lfr.to_numpy(),
    y_test_lfr.to_numpy(),
    prot_test_lfr.to_numpy()
)
inprocess_results['LFR']=fair_lfr_metrics

fair_lfr_metrics

{'accuracy': 0.9999555555555556,
 'disparate_impact': np.float64(0.9572306676095039),
 'statistical_parity_difference': np.float64(-0.04270676753913494),
 'equal_opportunity_difference': np.float64(0.0)}

In [29]:
print(np.unique(y_test_lfr, return_counts=True))

(array([0., 1.]), array([44908,    92]))


In [30]:
#reweighing

In [31]:
def apply_reweighing(X_train, y_train, prot_train):
    df_train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1), prot_train.reshape(-1, 1))),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(
        favorable_label=0,
        unfavorable_label=1,
        df=df_train,
        label_names=["label"],
        protected_attribute_names=["protected"]
    )
    RW = Reweighing(unprivileged_groups=[{'protected': 0}], privileged_groups=[{'protected': 1}])
    bld_rw = RW.fit_transform(bld_train)

    return bld_rw.features, bld_rw.labels.ravel(), bld_rw.instance_weights

In [32]:
X_train, X_test, y_train, y_test, prot_train, prot_test = preprocess_data(*load_gmsc_data(csv_path))

X_rw, y_rw, sample_weights = apply_reweighing(X_train, y_train, prot_train)

expected_num_features = X_rw.shape[1]

if X_test.shape[1] < expected_num_features:
    padding = expected_num_features - X_test.shape[1]
    X_test_aligned = np.hstack((X_test, np.zeros((X_test.shape[0], padding))))
elif X_test.shape[1] > expected_num_features:
    X_test_aligned = X_test[:, :expected_num_features]
else:
    X_test_aligned = X_test

model_rw = train_baseline_model(X_rw, y_rw, sample_weight=sample_weights)

reweighing_metric = evaluate_model(model_rw, X_test_aligned, y_test, prot_test)

inprocess_results['reweighing'] = reweighing_metric
reweighing_metric

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:38:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


{'accuracy': 0.9345333333333333,
 'disparate_impact': np.float64(1.0024113023945682),
 'statistical_parity_difference': np.float64(0.0023502519558527),
 'equal_opportunity_difference': np.float64(-0.0012591940035652227)}

In [33]:
for model, metrics in inprocess_results.items():
    print(f"🔹 {model}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")
    print()

🔹 Disparate_Impact_Remover
  accuracy: 0.9345
  disparate_impact: 0.9733
  statistical_parity_difference: -0.0262
  equal_opportunity_difference: -0.0196

🔹 LFR
  accuracy: 1.0000
  disparate_impact: 0.9572
  statistical_parity_difference: -0.0427
  equal_opportunity_difference: 0.0000

🔹 reweighing
  accuracy: 0.9345
  disparate_impact: 1.0024
  statistical_parity_difference: 0.0024
  equal_opportunity_difference: -0.0013



In [34]:
#Inprocess

In [35]:
X_train, X_test, y_train, y_test, prot_train, prot_test = load_gmsc_data_inprocess(csv_path)
results = {}

In [36]:
def train_gerryfair(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train[:, None], prot_train[:, None])),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train, label_names=["label"], protected_attribute_names=["protected"],
                                   favorable_label=1, unfavorable_label=0)

    df_test = pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test, label_names=["label"], protected_attribute_names=["protected"],
                                  favorable_label=0, unfavorable_label=1)

    clf = GerryFairClassifier(C=100, printflag=False, gamma=0.005, fairness_def='FP', max_iters=50)
    clf.fit(bld_train)
    pred = clf.predict(bld_test)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)

In [37]:
results['GerryFair'] = train_gerryfair(X_train, y_train, prot_train, X_test, y_test, prot_test)

In [38]:
def train_prejudice_remover(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1), prot_train.reshape(-1, 1))),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=0,
                                   unfavorable_label=1)

    df_test = pd.DataFrame(np.hstack((X_test, y_test.reshape(-1, 1), prot_test.reshape(-1, 1))),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=0,
                                  unfavorable_label=1)

    clf = PrejudiceRemover(sensitive_attr="protected", eta=25.0)
    clf.fit(bld_train)
    pred = clf.predict(bld_test)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)

In [39]:
results['PrejudiceRemover'] = train_prejudice_remover(X_train, y_train, prot_train, X_test, y_test, prot_test)

In [40]:
def train_expgrad(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train[:, None], prot_train[:, None])),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=0,
                                   unfavorable_label=1)

    df_test = pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=0,
                                  unfavorable_label=1)

    expgrad = ExponentiatedGradientReduction(
        estimator=LogisticRegression(solver='liblinear'),
        constraints="DemographicParity"
    )
    expgrad.fit(bld_train)

    pred = expgrad.predict(bld_test)
    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)

In [41]:
results['ExponentiatedGradient'] = train_expgrad(X_train, y_train, prot_train, X_test, y_test, prot_test)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [42]:
def train_gridsearch(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train[:, None], prot_train[:, None])),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=0,
                                   unfavorable_label=1)

    df_test = pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=0,
                                  unfavorable_label=1)

    grid = GridSearchReduction(
        estimator=LogisticRegression(solver='liblinear'),
        constraints="DemographicParity"
    )
    grid.fit(bld_train)

    pred = grid.predict(bld_test)
    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)

In [43]:
results['GridSearch'] = train_gridsearch(X_train, y_train, prot_train, X_test, y_test, prot_test)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [44]:
tf.disable_eager_execution()

In [45]:
def train_adversarial_debiasing(X_train, y_train, prot_train, X_test, y_test, prot_test):
    # Train data
    tf.reset_default_graph()
    df_train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1), prot_train.reshape(-1, 1))),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=0,
                                   unfavorable_label=1)

    # Test data
    df_test = pd.DataFrame(np.hstack((X_test, y_test.reshape(-1, 1), prot_test.reshape(-1, 1))),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=0,
                                  unfavorable_label=1)

    # TensorFlow session
    sess = tf.Session()

    clf = AdversarialDebiasing(
        privileged_groups=[{'protected': 1}],
        unprivileged_groups=[{'protected': 0}],
        scope_name='adv_debiasing',
        sess=sess,
        num_epochs=50,
        batch_size=64,
        debias=True
    )

    clf.fit(bld_train)
    pred = clf.predict(bld_test)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)

In [46]:
results['AdversarialDebiasing'] = train_adversarial_debiasing(X_train, y_train, prot_train, X_test, y_test, prot_test)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 0.599937; batch adversarial loss: 0.778756
epoch 0; iter: 200; batch classifier loss: 0.195469; batch adversarial loss: 0.544826
epoch 0; iter: 400; batch classifier loss: 0.361422; batch adversarial loss: 0.377997
epoch 0; iter: 600; batch classifier loss: 0.090559; batch adversarial loss: 0.271091
epoch 0; iter: 800; batch classifier loss: 0.273671; batch adversarial loss: 0.190935
epoch 0; iter: 1000; batch classifier loss: 0.140055; batch adversarial loss: 0.145991
epoch 0; iter: 1200; batch classifier loss: 0.267035; batch adversarial loss: 0.160065
epoch 0; iter: 1400; batch classifier loss: 0.130808; batch adversarial loss: 0.142225
epoch 0; iter: 1600; batch classifier loss: 0.176021; batch adversarial loss: 0.082687
epoch 1; iter: 0; batch classifier loss: 0.259216; batch adversarial loss: 0.129331
epoch 1; iter: 200; batch classifier loss: 0.164216; batch adversarial loss: 0.157572
epoch 1; iter: 400; batch classifier loss: 0.202294; b

In [47]:
print("Fairness Evaluation Across Compatible AIF360 In-processing Algorithms\n")
for model, metrics in results.items():
    print(f"🔹 {model}")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")
    print()

Fairness Evaluation Across Compatible AIF360 In-processing Algorithms

🔹 GerryFair
  accuracy: 0.9332
  disparate_impact: 0.9575
  statistical_parity_difference: -0.0424
  equal_opportunity_difference: -0.0313

🔹 PrejudiceRemover
  accuracy: 0.0659
  disparate_impact: 0.0000
  statistical_parity_difference: -0.0045
  equal_opportunity_difference: -0.0019

🔹 ExponentiatedGradient
  accuracy: 0.9340
  disparate_impact: 0.9880
  statistical_parity_difference: -0.0119
  equal_opportunity_difference: -0.0084

🔹 GridSearch
  accuracy: 0.9338
  disparate_impact: 1.0035
  statistical_parity_difference: 0.0034
  equal_opportunity_difference: 0.0015

🔹 AdversarialDebiasing
  accuracy: 0.9342
  disparate_impact: 1.0195
  statistical_parity_difference: 0.0188
  equal_opportunity_difference: 0.0100



In [48]:
#postprocessing

In [49]:
X_train, X_test, y_train, y_test, prot_train, prot_test = load_gmsc_data_inprocess(csv_path)
post_results = {}

In [50]:
def train_roc_postprocessing_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test):
    model = train_baseline_model(X_train, y_train)

    y_prob = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)

    bld_test = BinaryLabelDataset(
        favorable_label=0,
        unfavorable_label=1,
        df=pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                        columns=[f'x{i}' for i in range(X_test.shape[1])] + ['label', 'protected']),
        label_names=['label'],
        protected_attribute_names=['protected']
    )

    bld_pred = bld_test.copy()
    bld_pred.scores = y_prob.reshape(-1, 1)
    bld_pred.labels = y_pred.reshape(-1, 1)

    roc = RejectOptionClassification(
        unprivileged_groups=[{'protected': 0}],
        privileged_groups=[{'protected': 1}],
        low_class_thresh=0.3, high_class_thresh=0.7,
        num_class_thresh=100, num_ROC_margin=50,
        metric_name="Statistical parity difference",
        metric_ub=0.05, metric_lb=-0.05
    )
    roc = roc.fit(bld_test, bld_pred)
    pred = roc.predict(bld_pred)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)

In [51]:
post_results['RejectOptionClassification'] = train_roc_postprocessing_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:56:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [52]:
def train_calibrated_eq_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test):
    model = train_baseline_model(X_train, y_train)

    y_prob = model.predict_proba(X_test)[:, 1]

    bld_test = BinaryLabelDataset(
        favorable_label=0,
        unfavorable_label=1,
        df=pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                        columns=[f'x{i}' for i in range(X_test.shape[1])] + ['label', 'protected']),
        label_names=['label'],
        protected_attribute_names=['protected']
    )

    bld_pred = bld_test.copy()
    bld_pred.scores = y_prob.reshape(-1, 1)

    ceo = CalibratedEqOddsPostprocessing(
        privileged_groups=[{'protected': 1}],
        unprivileged_groups=[{'protected': 0}],
        cost_constraint="fnr",
        seed=42
    )
    ceo = ceo.fit(bld_test, bld_pred)
    pred = ceo.predict(bld_pred)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)

In [53]:
post_results['CalibratedEqOdds'] = train_calibrated_eq_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:57:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [54]:
def train_equalized_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test):
    model = train_baseline_model(X_train, y_train)

    y_pred = model.predict(X_test)

    bld_test = BinaryLabelDataset(
        favorable_label=0,
        unfavorable_label=1,
        df=pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                        columns=[f'x{i}' for i in range(X_test.shape[1])] + ['label', 'protected']),
        label_names=['label'],
        protected_attribute_names=['protected']
    )

    bld_pred = bld_test.copy()
    bld_pred.labels = y_pred.reshape(-1, 1)

    eq = EqOddsPostprocessing(
        privileged_groups=[{'protected': 1}],
        unprivileged_groups=[{'protected': 0}]
    )
    eq = eq.fit(bld_test, bld_pred)
    pred = eq.predict(bld_pred)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)

In [55]:
post_results['EqualizedOdds'] = train_equalized_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:57:43] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [56]:
for model, metrics in post_results.items():
    print(f"🔹 {model}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")
    print()

🔹 RejectOptionClassification
  accuracy: 0.0669
  disparate_impact: 2119.6431
  statistical_parity_difference: 0.0477
  equal_opportunity_difference: 0.0301

🔹 CalibratedEqOdds
  accuracy: 0.0646
  disparate_impact: 1.2066
  statistical_parity_difference: 0.0048
  equal_opportunity_difference: 0.0027

🔹 EqualizedOdds
  accuracy: 0.9289
  disparate_impact: 0.9935
  statistical_parity_difference: -0.0064
  equal_opportunity_difference: -0.0000

